# Building probabilistic Bayesian neural network models with TensorFlow Probability for Regression Problems

Based on the tutorial: https://keras.io/examples/keras_recipes/bayesian_neural_networks/

Taking a probabilistic approach to deep learning allows to account for **uncertainty**, so that models can assign less levels of confidence to incorrect predictions. Sources of uncertainty can be found in the data, due to measurement error or noise in the labels, or the model, due to insufficient data availability for the model to learn effectively.

This example demonstrates how to build basic **probabilistic Bayesian neural networks** to account for these **two types of uncertainty**. We use `TensorFlow Probability` library, which is compatible with Keras API and TensorFlow.

# Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# Create training and evaluation datasets

In [2]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

# Compile, train, and evaluate the model

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001

In [4]:
def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

# Create model inputs

In [5]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

In [6]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

# Experiment 1: standard neural network

In [7]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

In [9]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 37ms/step - loss: 42.4670 - root_mean_squared_error: 6.5167 - val_loss: 41.2288 - val_root_mean_squared_error: 6.4210
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 40.3187 - root_mean_squared_error: 6.3497 - val_loss: 39.6723 - val_root_mean_squared_error: 6.2986
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 38.4859 - root_mean_squared_error: 6.2037 - val_loss: 38.1149 - val_root_mean_squared_error: 6.1737
Epoch 4/100
17/17 [==============================] - 0s 7ms/step - loss: 36.7212 - root_mean_squared_error: 6.0598 - val_loss: 36.5522 - val_root_mean_squared_error: 6.0458
Epoch 5/100
17/17 [==============================] - 0s 7ms/step - loss: 34.9967 - root_mean_squared_error: 5.9158 - val_loss: 34.9804 - val_root_mean_squared_error: 5.9144
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 33.3076 - root_mean_squared_error

Epoch 96/100
17/17 [==============================] - 0s 7ms/step - loss: 0.6768 - root_mean_squared_error: 0.8227 - val_loss: 0.6923 - val_root_mean_squared_error: 0.8321
Epoch 97/100
17/17 [==============================] - 0s 7ms/step - loss: 0.6713 - root_mean_squared_error: 0.8193 - val_loss: 0.6846 - val_root_mean_squared_error: 0.8274
Epoch 98/100
17/17 [==============================] - 0s 7ms/step - loss: 0.6674 - root_mean_squared_error: 0.8170 - val_loss: 0.6795 - val_root_mean_squared_error: 0.8243
Epoch 99/100
17/17 [==============================] - 0s 7ms/step - loss: 0.6626 - root_mean_squared_error: 0.8140 - val_loss: 0.6740 - val_root_mean_squared_error: 0.8210
Epoch 100/100
17/17 [==============================] - 0s 7ms/step - loss: 0.6601 - root_mean_squared_error: 0.8125 - val_loss: 0.6697 - val_root_mean_squared_error: 0.8183
Model training finished.
Train RMSE: 0.809
Evaluating model performance...
Test RMSE: 0.818


In [10]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[0]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.1 - Actual: 6.0
Predicted: 5.9 - Actual: 6.0
Predicted: 6.0 - Actual: 6.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.9 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 6.2 - Actual: 6.0
Predicted: 6.2 - Actual: 6.0
Predicted: 6.1 - Actual: 8.0
Predicted: 6.1 - Actual: 6.0


# Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.

In [11]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model

In [12]:
# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

We use the `tfp.layers.DenseVariational` layer instead of the standard `keras.layers.Dense` layer in the neural network model.

In [13]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.

In [16]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

## Train BNN with a small training subset

In [14]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

In [15]:
bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 247ms/step - loss: 30.2974 - root_mean_squared_error: 5.5030 - val_loss: 27.5546 - val_root_mean_squared_error: 5.2481
Epoch 2/500
5/5 [==============================] - 0s 24ms/step - loss: 29.4284 - root_mean_squared_error: 5.4233 - val_loss: 26.7680 - val_root_mean_squared_error: 5.1724
Epoch 3/500
5/5 [==============================] - 0s 25ms/step - loss: 28.1872 - root_mean_squared_error: 5.3078 - val_loss: 27.0261 - val_root_mean_squared_error: 5.1972
Epoch 4/500
5/5 [==============================] - 0s 26ms/step - loss: 28.0096 - root_mean_squared_error: 5.2910 - val_loss: 25.8849 - val_root_mean_squared_error: 5.0860
Epoch 5/500
5/5 [==============================] - 0s 23ms/step - loss: 27.3345 - root_mean_squared_error: 5.2267 - val_loss: 24.4275 - val_root_mean_squared_error: 4.9410
Epoch 6/500
5/5 [==============================] - 0s 25ms/step - loss: 25.6413 - root_mean_squared_error: 5.06

Epoch 96/500
5/5 [==============================] - 0s 23ms/step - loss: 2.9299 - root_mean_squared_error: 1.7054 - val_loss: 2.3719 - val_root_mean_squared_error: 1.5330
Epoch 97/500
5/5 [==============================] - 0s 24ms/step - loss: 2.5818 - root_mean_squared_error: 1.6004 - val_loss: 2.0992 - val_root_mean_squared_error: 1.4411
Epoch 98/500
5/5 [==============================] - 0s 25ms/step - loss: 2.6600 - root_mean_squared_error: 1.6248 - val_loss: 3.1407 - val_root_mean_squared_error: 1.7673
Epoch 99/500
5/5 [==============================] - 0s 24ms/step - loss: 2.7337 - root_mean_squared_error: 1.6471 - val_loss: 2.3453 - val_root_mean_squared_error: 1.5237
Epoch 100/500
5/5 [==============================] - 0s 25ms/step - loss: 2.8799 - root_mean_squared_error: 1.6904 - val_loss: 2.6236 - val_root_mean_squared_error: 1.6137
Epoch 101/500
5/5 [==============================] - 0s 23ms/step - loss: 2.4327 - root_mean_squared_error: 1.5533 - val_loss: 2.6850 - val_root

Epoch 144/500
5/5 [==============================] - 0s 23ms/step - loss: 0.7790 - root_mean_squared_error: 0.8664 - val_loss: 0.8273 - val_root_mean_squared_error: 0.8947
Epoch 145/500
5/5 [==============================] - 0s 24ms/step - loss: 0.8291 - root_mean_squared_error: 0.8964 - val_loss: 0.8367 - val_root_mean_squared_error: 0.9000
Epoch 146/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7969 - root_mean_squared_error: 0.8792 - val_loss: 0.7974 - val_root_mean_squared_error: 0.8781
Epoch 147/500
5/5 [==============================] - 0s 26ms/step - loss: 0.7662 - root_mean_squared_error: 0.8596 - val_loss: 0.8404 - val_root_mean_squared_error: 0.9025
Epoch 148/500
5/5 [==============================] - 0s 23ms/step - loss: 0.7808 - root_mean_squared_error: 0.8679 - val_loss: 0.8338 - val_root_mean_squared_error: 0.8989
Epoch 149/500
5/5 [==============================] - 0s 24ms/step - loss: 0.8355 - root_mean_squared_error: 0.9001 - val_loss: 0.7715 - val_

Epoch 239/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7274 - root_mean_squared_error: 0.8370 - val_loss: 0.7475 - val_root_mean_squared_error: 0.8497
Epoch 240/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7116 - root_mean_squared_error: 0.8265 - val_loss: 0.7380 - val_root_mean_squared_error: 0.8426
Epoch 241/500
5/5 [==============================] - 0s 23ms/step - loss: 0.7475 - root_mean_squared_error: 0.8482 - val_loss: 0.7681 - val_root_mean_squared_error: 0.8624
Epoch 242/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7679 - root_mean_squared_error: 0.8607 - val_loss: 0.7218 - val_root_mean_squared_error: 0.8335
Epoch 243/500
5/5 [==============================] - 0s 26ms/step - loss: 0.6684 - root_mean_squared_error: 0.7994 - val_loss: 0.7215 - val_root_mean_squared_error: 0.8310
Epoch 244/500
5/5 [==============================] - 0s 27ms/step - loss: 0.7111 - root_mean_squared_error: 0.8284 - val_loss: 0.7604 - val_

Epoch 334/500
5/5 [==============================] - 0s 25ms/step - loss: 0.6300 - root_mean_squared_error: 0.7740 - val_loss: 0.6711 - val_root_mean_squared_error: 0.7996
Epoch 335/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6287 - root_mean_squared_error: 0.7762 - val_loss: 0.6566 - val_root_mean_squared_error: 0.7928
Epoch 336/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7789 - root_mean_squared_error: 0.8657 - val_loss: 0.6943 - val_root_mean_squared_error: 0.8169
Epoch 337/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7045 - root_mean_squared_error: 0.8210 - val_loss: 0.6790 - val_root_mean_squared_error: 0.8081
Epoch 338/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7308 - root_mean_squared_error: 0.8378 - val_loss: 0.6966 - val_root_mean_squared_error: 0.8173
Epoch 339/500
5/5 [==============================] - 0s 25ms/step - loss: 0.7604 - root_mean_squared_error: 0.8547 - val_loss: 0.6890 - val_

Epoch 429/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6578 - root_mean_squared_error: 0.7923 - val_loss: 0.6832 - val_root_mean_squared_error: 0.8106
Epoch 430/500
5/5 [==============================] - 0s 24ms/step - loss: 0.7249 - root_mean_squared_error: 0.8349 - val_loss: 0.6342 - val_root_mean_squared_error: 0.7778
Epoch 431/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6572 - root_mean_squared_error: 0.7943 - val_loss: 0.6544 - val_root_mean_squared_error: 0.7902
Epoch 432/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6768 - root_mean_squared_error: 0.8034 - val_loss: 0.7288 - val_root_mean_squared_error: 0.8358
Epoch 433/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6570 - root_mean_squared_error: 0.7925 - val_loss: 0.6796 - val_root_mean_squared_error: 0.8062
Epoch 434/500
5/5 [==============================] - 0s 24ms/step - loss: 0.6625 - root_mean_squared_error: 0.7951 - val_loss: 0.6604 - val_

In [17]:
compute_predictions(bnn_model_small)

Predictions mean: 5.88, min: 5.19, max: 6.25, range: 1.06 - Actual: 6.0
Predictions mean: 6.02, min: 5.61, max: 6.33, range: 0.71 - Actual: 6.0
Predictions mean: 5.58, min: 4.73, max: 6.08, range: 1.35 - Actual: 6.0
Predictions mean: 5.9, min: 5.34, max: 6.21, range: 0.87 - Actual: 6.0
Predictions mean: 5.09, min: 4.1, max: 5.65, range: 1.55 - Actual: 5.0
Predictions mean: 6.27, min: 5.76, max: 6.47, range: 0.71 - Actual: 6.0
Predictions mean: 6.26, min: 5.78, max: 6.45, range: 0.67 - Actual: 6.0
Predictions mean: 6.27, min: 5.88, max: 6.46, range: 0.58 - Actual: 6.0
Predictions mean: 6.09, min: 5.74, max: 6.34, range: 0.6 - Actual: 8.0
Predictions mean: 6.24, min: 5.73, max: 6.44, range: 0.7 - Actual: 6.0


## Train BNN with the whole training set

In [18]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)

In [19]:
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 42ms/step - loss: 32.5822 - root_mean_squared_error: 5.7077 - val_loss: 28.1492 - val_root_mean_squared_error: 5.3053
Epoch 2/500
17/17 [==============================] - 0s 7ms/step - loss: 31.2286 - root_mean_squared_error: 5.5879 - val_loss: 30.3695 - val_root_mean_squared_error: 5.5104
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 29.9602 - root_mean_squared_error: 5.4731 - val_loss: 28.3165 - val_root_mean_squared_error: 5.3208
Epoch 4/500
17/17 [==============================] - 0s 8ms/step - loss: 29.1262 - root_mean_squared_error: 5.3965 - val_loss: 27.7897 - val_root_mean_squared_error: 5.2711
Epoch 5/500
17/17 [==============================] - 0s 7ms/step - loss: 26.4684 - root_mean_squared_error: 5.1443 - val_loss: 26.1980 - val_root_mean_squared_error: 5.1180
Epoch 6/500
17/17 [==============================] - 0s 8ms/step - loss: 26.4781 - root_mean_squared_error

Epoch 96/500
17/17 [==============================] - 0s 8ms/step - loss: 0.6612 - root_mean_squared_error: 0.8063 - val_loss: 0.6291 - val_root_mean_squared_error: 0.7859
Epoch 97/500
17/17 [==============================] - 0s 8ms/step - loss: 0.6603 - root_mean_squared_error: 0.8061 - val_loss: 0.6669 - val_root_mean_squared_error: 0.8097
Epoch 98/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6639 - root_mean_squared_error: 0.8078 - val_loss: 0.6877 - val_root_mean_squared_error: 0.8229
Epoch 99/500
17/17 [==============================] - 0s 8ms/step - loss: 0.6560 - root_mean_squared_error: 0.8032 - val_loss: 0.6259 - val_root_mean_squared_error: 0.7848
Epoch 100/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6521 - root_mean_squared_error: 0.8011 - val_loss: 0.6376 - val_root_mean_squared_error: 0.7917
Epoch 101/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6514 - root_mean_squared_error: 0.8008 - val_loss: 0.6540 - va

17/17 [==============================] - 0s 7ms/step - loss: 0.6139 - root_mean_squared_error: 0.7757 - val_loss: 0.6159 - val_root_mean_squared_error: 0.7768
Epoch 191/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6114 - root_mean_squared_error: 0.7737 - val_loss: 0.6042 - val_root_mean_squared_error: 0.7694
Epoch 192/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6231 - root_mean_squared_error: 0.7814 - val_loss: 0.6118 - val_root_mean_squared_error: 0.7744
Epoch 193/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6222 - root_mean_squared_error: 0.7808 - val_loss: 0.6235 - val_root_mean_squared_error: 0.7818
Epoch 194/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6206 - root_mean_squared_error: 0.7799 - val_loss: 0.6100 - val_root_mean_squared_error: 0.7730
Epoch 195/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6138 - root_mean_squared_error: 0.7753 - val_loss: 0.6279 - val_root_mea

17/17 [==============================] - 0s 7ms/step - loss: 0.6006 - root_mean_squared_error: 0.7660 - val_loss: 0.6029 - val_root_mean_squared_error: 0.7677
Epoch 285/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6247 - root_mean_squared_error: 0.7816 - val_loss: 0.6298 - val_root_mean_squared_error: 0.7846
Epoch 286/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6030 - root_mean_squared_error: 0.7676 - val_loss: 0.5912 - val_root_mean_squared_error: 0.7603
Epoch 287/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6119 - root_mean_squared_error: 0.7735 - val_loss: 0.6036 - val_root_mean_squared_error: 0.7686
Epoch 288/500
17/17 [==============================] - 0s 7ms/step - loss: 0.5993 - root_mean_squared_error: 0.7654 - val_loss: 0.6090 - val_root_mean_squared_error: 0.7717
Epoch 289/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6087 - root_mean_squared_error: 0.7711 - val_loss: 0.6323 - val_root_mea

17/17 [==============================] - 0s 7ms/step - loss: 0.6120 - root_mean_squared_error: 0.7729 - val_loss: 0.6045 - val_root_mean_squared_error: 0.7683
Epoch 379/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6044 - root_mean_squared_error: 0.7682 - val_loss: 0.5822 - val_root_mean_squared_error: 0.7545
Epoch 380/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6026 - root_mean_squared_error: 0.7671 - val_loss: 0.6190 - val_root_mean_squared_error: 0.7778
Epoch 381/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6084 - root_mean_squared_error: 0.7710 - val_loss: 0.5920 - val_root_mean_squared_error: 0.7601
Epoch 382/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6090 - root_mean_squared_error: 0.7711 - val_loss: 0.6003 - val_root_mean_squared_error: 0.7656
Epoch 383/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6079 - root_mean_squared_error: 0.7702 - val_loss: 0.6042 - val_root_mea

17/17 [==============================] - 0s 14ms/step - loss: 0.5951 - root_mean_squared_error: 0.7618 - val_loss: 0.5980 - val_root_mean_squared_error: 0.7639
Epoch 473/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6022 - root_mean_squared_error: 0.7667 - val_loss: 0.6128 - val_root_mean_squared_error: 0.7726
Epoch 474/500
17/17 [==============================] - 0s 7ms/step - loss: 0.5910 - root_mean_squared_error: 0.7591 - val_loss: 0.5810 - val_root_mean_squared_error: 0.7518
Epoch 475/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6120 - root_mean_squared_error: 0.7728 - val_loss: 0.5944 - val_root_mean_squared_error: 0.7609
Epoch 476/500
17/17 [==============================] - 0s 7ms/step - loss: 0.5980 - root_mean_squared_error: 0.7638 - val_loss: 0.6071 - val_root_mean_squared_error: 0.7687
Epoch 477/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6084 - root_mean_squared_error: 0.7708 - val_loss: 0.5860 - val_root_me

In [20]:
compute_predictions(bnn_model_full)

Predictions mean: 5.91, min: 5.47, max: 6.25, range: 0.77 - Actual: 6.0
Predictions mean: 6.02, min: 5.78, max: 6.26, range: 0.48 - Actual: 6.0
Predictions mean: 5.64, min: 5.22, max: 6.11, range: 0.9 - Actual: 6.0
Predictions mean: 5.89, min: 5.62, max: 6.14, range: 0.52 - Actual: 6.0
Predictions mean: 5.21, min: 4.91, max: 5.73, range: 0.82 - Actual: 5.0
Predictions mean: 6.47, min: 6.23, max: 6.59, range: 0.36 - Actual: 6.0
Predictions mean: 6.47, min: 6.22, max: 6.6, range: 0.38 - Actual: 6.0
Predictions mean: 6.4, min: 6.12, max: 6.52, range: 0.39 - Actual: 6.0
Predictions mean: 6.18, min: 5.9, max: 6.35, range: 0.45 - Actual: 8.0
Predictions mean: 6.36, min: 6.08, max: 6.55, range: 0.48 - Actual: 6.0


Notice that the model trained with the full training dataset shows smaller range (uncertainty) in the prediction values for the same inputs, compared to the model trained with a subset of the training dataset.

# Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the **aleatoric uncertainty as well**, which is due to **irreducible noise in the data**, or to the **stochastic nature of the process generating the data**.

In this **regression** example, we model the output as an `IndependentNormal` distribution, with learnable mean and variance parameters. If the task was **classification**, we would have used `IndependentBernoulli` with *binary classes*, and `OneHotCategorical` with *multiple classes*, to model distribution of the model output.

In [21]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since **the output of the model is a distribution**, rather than a point estimate, we use the `negative loglikelihood` as our **loss function** to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [22]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

In [23]:
num_epochs = 1000

In [24]:
prob_bnn_model = create_probablistic_bnn_model(train_size)

In [25]:
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 47ms/step - loss: 16.1739 - root_mean_squared_error: 6.0463 - val_loss: 17.0367 - val_root_mean_squared_error: 5.8139
Epoch 2/1000
17/17 [==============================] - 0s 8ms/step - loss: 15.0467 - root_mean_squared_error: 5.9704 - val_loss: 12.9173 - val_root_mean_squared_error: 5.7350
Epoch 3/1000
17/17 [==============================] - 0s 8ms/step - loss: 13.6312 - root_mean_squared_error: 5.9511 - val_loss: 11.7350 - val_root_mean_squared_error: 5.9694
Epoch 4/1000
17/17 [==============================] - 0s 8ms/step - loss: 12.8354 - root_mean_squared_error: 5.8179 - val_loss: 9.0928 - val_root_mean_squared_error: 5.7054
Epoch 5/1000
17/17 [==============================] - 0s 8ms/step - loss: 9.0729 - root_mean_squared_error: 5.5928 - val_loss: 10.9736 - val_root_mean_squared_error: 5.6963
Epoch 6/1000
17/17 [==============================] - 0s 8ms/step - loss: 9.8999 - root_mean_squared_er

17/17 [==============================] - 0s 7ms/step - loss: 1.2440 - root_mean_squared_error: 1.1749 - val_loss: 1.2038 - val_root_mean_squared_error: 1.1510
Epoch 96/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2403 - root_mean_squared_error: 1.1651 - val_loss: 1.2458 - val_root_mean_squared_error: 1.1798
Epoch 97/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2380 - root_mean_squared_error: 1.1677 - val_loss: 1.2328 - val_root_mean_squared_error: 1.1390
Epoch 98/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2270 - root_mean_squared_error: 1.1820 - val_loss: 1.2196 - val_root_mean_squared_error: 1.1250
Epoch 99/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2252 - root_mean_squared_error: 1.1592 - val_loss: 1.2243 - val_root_mean_squared_error: 1.1765
Epoch 100/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.2373 - root_mean_squared_error: 1.1746 - val_loss: 1.2042 - val_root_me

17/17 [==============================] - 0s 8ms/step - loss: 1.1792 - root_mean_squared_error: 1.1005 - val_loss: 1.1618 - val_root_mean_squared_error: 1.0573
Epoch 190/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1728 - root_mean_squared_error: 1.0809 - val_loss: 1.1829 - val_root_mean_squared_error: 1.1591
Epoch 191/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1790 - root_mean_squared_error: 1.1060 - val_loss: 1.1664 - val_root_mean_squared_error: 1.0871
Epoch 192/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.2013 - root_mean_squared_error: 1.1370 - val_loss: 1.2043 - val_root_mean_squared_error: 1.1534
Epoch 193/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1798 - root_mean_squared_error: 1.1074 - val_loss: 1.2053 - val_root_mean_squared_error: 1.0835
Epoch 194/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1895 - root_mean_squared_error: 1.1115 - val_loss: 1.1739 - val_roo

17/17 [==============================] - 0s 7ms/step - loss: 1.1755 - root_mean_squared_error: 1.0977 - val_loss: 1.1680 - val_root_mean_squared_error: 1.0544
Epoch 284/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1747 - root_mean_squared_error: 1.0860 - val_loss: 1.1789 - val_root_mean_squared_error: 1.0921
Epoch 285/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1683 - root_mean_squared_error: 1.1045 - val_loss: 1.1506 - val_root_mean_squared_error: 1.0533
Epoch 286/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1675 - root_mean_squared_error: 1.0768 - val_loss: 1.1642 - val_root_mean_squared_error: 1.0659
Epoch 287/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1786 - root_mean_squared_error: 1.1019 - val_loss: 1.1629 - val_root_mean_squared_error: 1.0919
Epoch 288/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1755 - root_mean_squared_error: 1.0916 - val_loss: 1.1650 - val_roo

17/17 [==============================] - 0s 8ms/step - loss: 1.1634 - root_mean_squared_error: 1.0708 - val_loss: 1.1499 - val_root_mean_squared_error: 1.0669
Epoch 378/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1664 - root_mean_squared_error: 1.0993 - val_loss: 1.1485 - val_root_mean_squared_error: 1.0632
Epoch 379/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1490 - root_mean_squared_error: 1.0777 - val_loss: 1.1592 - val_root_mean_squared_error: 1.0586
Epoch 380/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1642 - root_mean_squared_error: 1.0876 - val_loss: 1.1806 - val_root_mean_squared_error: 1.0888
Epoch 381/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1575 - root_mean_squared_error: 1.0820 - val_loss: 1.1591 - val_root_mean_squared_error: 1.1029
Epoch 382/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1561 - root_mean_squared_error: 1.0793 - val_loss: 1.1550 - val_roo

...

17/17 [==============================] - 0s 8ms/step - loss: 1.1611 - root_mean_squared_error: 1.0593 - val_loss: 1.1528 - val_root_mean_squared_error: 1.0455
Epoch 566/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1617 - root_mean_squared_error: 1.0946 - val_loss: 1.1775 - val_root_mean_squared_error: 1.0819
Epoch 567/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1636 - root_mean_squared_error: 1.0987 - val_loss: 1.1613 - val_root_mean_squared_error: 1.1099
Epoch 568/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1619 - root_mean_squared_error: 1.0721 - val_loss: 1.1989 - val_root_mean_squared_error: 1.0942
Epoch 569/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1656 - root_mean_squared_error: 1.0962 - val_loss: 1.1543 - val_root_mean_squared_error: 1.0954
Epoch 570/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1675 - root_mean_squared_error: 1.0780 - val_loss: 1.1853 - val_roo

17/17 [==============================] - 0s 8ms/step - loss: 1.1565 - root_mean_squared_error: 1.0727 - val_loss: 1.1624 - val_root_mean_squared_error: 1.0592
Epoch 660/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1536 - root_mean_squared_error: 1.0715 - val_loss: 1.1657 - val_root_mean_squared_error: 1.0544
Epoch 661/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1577 - root_mean_squared_error: 1.0649 - val_loss: 1.1586 - val_root_mean_squared_error: 1.0128
Epoch 662/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1575 - root_mean_squared_error: 1.0530 - val_loss: 1.1795 - val_root_mean_squared_error: 1.0950
Epoch 663/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1527 - root_mean_squared_error: 1.0876 - val_loss: 1.1663 - val_root_mean_squared_error: 1.0665
Epoch 664/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1670 - root_mean_squared_error: 1.0875 - val_loss: 1.1860 - val_roo

17/17 [==============================] - 0s 7ms/step - loss: 1.1617 - root_mean_squared_error: 1.0801 - val_loss: 1.1482 - val_root_mean_squared_error: 1.0209
Epoch 754/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1550 - root_mean_squared_error: 1.0870 - val_loss: 1.1477 - val_root_mean_squared_error: 1.1142
Epoch 755/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1539 - root_mean_squared_error: 1.0806 - val_loss: 1.1418 - val_root_mean_squared_error: 1.1023
Epoch 756/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1586 - root_mean_squared_error: 1.0737 - val_loss: 1.1655 - val_root_mean_squared_error: 1.1247
Epoch 757/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1551 - root_mean_squared_error: 1.0833 - val_loss: 1.1450 - val_root_mean_squared_error: 1.0532
Epoch 758/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1541 - root_mean_squared_error: 1.0767 - val_loss: 1.1660 - val_roo

17/17 [==============================] - 0s 9ms/step - loss: 1.1467 - root_mean_squared_error: 1.0666 - val_loss: 1.1515 - val_root_mean_squared_error: 1.0911
Epoch 848/1000
17/17 [==============================] - 0s 13ms/step - loss: 1.1450 - root_mean_squared_error: 1.0443 - val_loss: 1.1509 - val_root_mean_squared_error: 1.0524
Epoch 849/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1514 - root_mean_squared_error: 1.0780 - val_loss: 1.1515 - val_root_mean_squared_error: 1.0913
Epoch 850/1000
17/17 [==============================] - 0s 15ms/step - loss: 1.1546 - root_mean_squared_error: 1.0590 - val_loss: 1.1577 - val_root_mean_squared_error: 1.0719
Epoch 851/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1549 - root_mean_squared_error: 1.0844 - val_loss: 1.1597 - val_root_mean_squared_error: 1.0606
Epoch 852/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1542 - root_mean_squared_error: 1.0422 - val_loss: 1.1569 - val

17/17 [==============================] - 0s 8ms/step - loss: 1.1444 - root_mean_squared_error: 1.0876 - val_loss: 1.1487 - val_root_mean_squared_error: 1.0701
Epoch 942/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1485 - root_mean_squared_error: 1.0484 - val_loss: 1.1507 - val_root_mean_squared_error: 1.0457
Epoch 943/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1524 - root_mean_squared_error: 1.0938 - val_loss: 1.1502 - val_root_mean_squared_error: 1.0140
Epoch 944/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1428 - root_mean_squared_error: 1.0692 - val_loss: 1.1665 - val_root_mean_squared_error: 1.0835
Epoch 945/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1425 - root_mean_squared_error: 1.0551 - val_loss: 1.1479 - val_root_mean_squared_error: 1.0332
Epoch 946/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1439 - root_mean_squared_error: 1.0805 - val_loss: 1.1454 - val_roo

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the **confidence intervals (CI)** of the prediction.

In [26]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

In [27]:
# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

In [28]:
for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.96, stddev: 0.76, 95% CI: [7.45 - 4.47] - Actual: 6.0
Prediction mean: 5.78, stddev: 0.74, 95% CI: [7.23 - 4.33] - Actual: 6.0
Prediction mean: 5.6, stddev: 0.72, 95% CI: [7.01 - 4.19] - Actual: 6.0
Prediction mean: 5.64, stddev: 0.73, 95% CI: [7.06 - 4.21] - Actual: 6.0
Prediction mean: 5.24, stddev: 0.68, 95% CI: [6.57 - 3.9] - Actual: 5.0
Prediction mean: 6.4, stddev: 0.81, 95% CI: [7.99 - 4.81] - Actual: 6.0
Prediction mean: 6.31, stddev: 0.8, 95% CI: [7.87 - 4.74] - Actual: 6.0
Prediction mean: 6.16, stddev: 0.78, 95% CI: [7.68 - 4.63] - Actual: 6.0
Prediction mean: 5.96, stddev: 0.76, 95% CI: [7.45 - 4.47] - Actual: 8.0
Prediction mean: 6.22, stddev: 0.79, 95% CI: [7.77 - 4.68] - Actual: 6.0
